#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 05 - Containers 1 - Exercise Solutions</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

### 01 Exercises

This exercise will cover creating a simple Docker application for doing some simple file operations. We'll cover building, running, interacting with the host file system, managing running containers, and other admins operations.

### Installation

The first thing is install Docker if you don't already have it. 

* **Linux**: Most of our workstations are Ubuntu so follow the instructions [here](https://docs.docker.com/engine/install/ubuntu). Other Linuces (Linuxes?) are [supported](https://docs.docker.com/engine/install/#server) but might be harder to trouble shoot.


* **Windows**: Instructions are [here](https://docs.docker.com/docker-for-windows/install). This should be a simple application installation, just be sure to [switch to Linux containers](https://docs.docker.com/docker-for-windows/#switch-between-windows-and-linux-containers).


* **macOS**: Instructions are [here](https://docs.docker.com/docker-for-mac/install). This should also be straight-forward, however I (Eric) don't have a Mac with me so it might be difficult trouble shooting if anything comes up, using another machine or logging remotely into a Ubuntu workstation on the KCL network may be better.

### Exercise 1: 

Let's create a Docker app which inverts the colours of images in a folder, placing the results in "inverted" subfolder. We'll use [ImageMagick](https://imagemagick.org) and a simple bash script to do this. 

**Step 1:** Create a directory called `exercise01` and copy the following into it as `invert.sh`:

```sh
#! /bin/bash

# files are read from this directory
INPUT_DIR=/input

# create the output directory
mkdir -p $INPUT_DIR/inverted

# iterate over every file with an extension in INPUT_DIR and convert it, this will fail for non-images of course
for i in $INPUT_DIR/*.*
do
    # if $i is a file attempt to convert it
    if [ -f "$i" ]
    then
        outfile=$INPUT_DIR/inverted/$(basename "$i")
        convert -channel RGB -negate "$i" "$outfile"
    fi
done
```

In [2]:
!mkdir -p exercise01

In [3]:
%%writefile exercise01/invert.sh

#! /bin/bash

# files are read from this directory
INPUT_DIR=/input

# create the output directory
mkdir -p $INPUT_DIR/inverted

# iterate over every file with an extension in INPUT_DIR and convert it, this will fail for non-images of course
for i in $INPUT_DIR/*.*
do
    # if $i is a file attempt to convert it
    if [ -f "$i" ]
    then
        outfile=$INPUT_DIR/inverted/$(basename "$i")
        convert -channel RGB -negate "$i" "$outfile"
    fi
done

Writing exercise01/invert.sh


**Step 2:** We now want to define a `Dockerfile` in the `exercise01` directory. This file will need to do the following:

* Specify the base image (I'd suggest some flavour of Ubuntu)

* Install ImageMagick (use `apt` for this, although you could download the pre-built `magick` executable from their site and use `magick convert`)

* Copy the script to the root directory

* Set the command to run the script when the container is run

Create and fill in your `Dockerfile` to accomplish this, the reference for commands is [here](https://docs.docker.com/engine/reference/builder).

In [4]:
%%writefile exercise01/Dockerfile

FROM ubuntu:18.04
RUN apt update --fix-missing
RUN apt install imagemagick -y

COPY invert.sh /

CMD ["/bin/bash","invert.sh"]

Writing exercise01/Dockerfile


**Step 3:** Build your image using the `docker build` command, choosing to tag it as `image-invert`. If you need help on the command `docker build --help` will list options and usage information.

One reason why we've created a directory to do the building in is that Docker will copy everything in a build directory to a temporary location, so having lots of large files where your Dockerfile lives isn't a good idea.

In [4]:
!docker build exercise01 -t image-invert

Sending build context to Docker daemon  3.072kB
Step 1/5 : from ubuntu
 ---> a2a15febcdf3
Step 2/5 : RUN apt update
 ---> Using cache
 ---> 1c6378934397
Step 3/5 : RUN apt install imagemagick -y
 ---> Using cache
 ---> 9e8586e26a6f
Step 4/5 : COPY invert.sh /
 ---> Using cache
 ---> 18910163daa0
Step 5/5 : CMD ["/bin/bash","invert.sh"]
 ---> Using cache
 ---> 62b1c9691ca8
Successfully built 62b1c9691ca8
Successfully tagged image-invert:latest


**Step 4:** Now we want to run our app and convert some images. In the workshop directory we have the directory `testimages`, but how do we give the container access to this directory to read from and write to? The script mentions a directory called `/input` for reading and writing, but that doesn't exist in the image.

One way external data is accessed in Docker is through binding volumes, which in our case making a local directory accessible under a given name. The documentation on this is [here](https://docs.docker.com/storage/bind-mounts/#start-a-container-with-a-bind-mount). 

Run your Docker image from the workshop directory with the added argument `-v /full/path/to/testimages:/input`. This creates a volume binding associating `/full/path/to/testimages` (ie. the absolute path to `testimages`) to `/input` in the container. After running your container check `testimages` to see what's there now.

The alternative and more current syntax for command line arguments to do binding is with `--bind`, for us this would look like `--mount src=/full/path/to/testimages,target=/input,type=bind`.

In [5]:
!docker run --rm -v $(pwd)/testimages:/input image-invert
!ls -l testimages/inverted

total 368
-rw-r--r-- 1 root root  30553 May 29 18:21 cat00.jpg
-rw-r--r-- 1 root root  12352 May 29 18:21 cat01.jpg
-rw-r--r-- 1 root root 107932 May 29 18:21 cat02.jpg
-rw-r--r-- 1 root root 117656 May 29 18:21 cat03.jpg
-rw-r--r-- 1 root root  96046 May 29 18:21 cat04.jpg


**Step 5:** If you're running on Linux you'll notice that the created directory in `testimages` is owned by root. This is because the Docker daemon runs as root and the command within the container also runs as root. It's not good or secure practice to allow a container command to run internally as root so we have to create an internal user account and run as that.

Delete the output directory using `sudo rm -rf testimages/inverted`.

In your `Dockerfile` before the `CMD` line at the end, add the following:

```dockerfile
RUN adduser dockeruser --shell /bin/bash
USER dockeruser
```

What these commands do is run `adduser` to create an internal user account, then `USER` states the following commands will be run as that user in the container. Now the script file will be run as `dockeruser`, whose user ID (UID) and group ID (GID) don't match anything on the host system. 

Using a non-root user internally, one without elevated privileges, has solved our security concern but not the file ownership one. Attempting to run the container now with the same command will result in error if `testimages` doesn't have permissions set to allow any user to write to it (which is the default).

The solution now is to use the `--user`/`-u` flag to specify the user under which to run the container. The syntax we'll use with this flag is `UID:GID` which you can get with the `id` command, so add `-u $(id -u):$(id -g)` to your `docker run` command and run the container again. Your generated images should be owned by you now.

In [5]:
%%writefile exercise01/Dockerfile

FROM ubuntu:18.04
RUN apt update --fix-missing
RUN apt install imagemagick -y

COPY invert.sh /

RUN adduser dockeruser --shell /bin/bash
USER dockeruser

CMD ["/bin/bash","invert.sh"]

Overwriting exercise01/Dockerfile


In [6]:
!docker build exercise01 -t image-invert

Sending build context to Docker daemon  3.072kB
Step 1/7 : FROM ubuntu:18.04
 ---> a2a15febcdf3
Step 2/7 : RUN apt update --fix-missing
 ---> Running in d90ba4d72daa


Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [9092 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1253 kB]
Get:11 http://security.ubuntu.com/ubuntu bioni

Get:28 http://archive.ubuntu.com/ubuntu bionic/main amd64 liblqr-1-0 amd64 0.4.2-2.1 [27.7 kB]
Get:29 http://archive.ubuntu.com/ubuntu bionic/main amd64 libltdl7 amd64 2.4.6-2 [38.8 kB]
Get:30 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libtiff5 amd64 4.0.9-5ubuntu0.3 [153 kB]
Get:31 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libicu60 amd64 60.2-3ubuntu3.1 [8054 kB]
Get:32 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libxml2 amd64 2.9.4+dfsg1-6.1ubuntu1.3 [663 kB]
Get:33 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 imagemagick-6-common all 8:6.9.7.4+dfsg-16ubuntu6.8 [60.0 kB]
Get:34 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagickcore-6.q16-3 amd64 8:6.9.7.4+dfsg-16ubuntu6.8 [1616 kB]
Get:35 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagickwand-6.q16-3 amd64 8:6.9.7.4+dfsg-16ubuntu6.8 [294 kB]
Get:36 http://archive.ubuntu.com/ubuntu bionic/main amd64 poppler-data all 0.4.8-2 [1479 kB]
Get:

Selecting previously unselected package libexpat1:amd64.
Preparing to unpack .../07-libexpat1_2.2.5-3ubuntu0.2_amd64.deb ...
Unpacking libexpat1:amd64 (2.2.5-3ubuntu0.2) ...
Selecting previously unselected package libpng16-16:amd64.
Preparing to unpack .../08-libpng16-16_1.6.34-1ubuntu0.18.04.2_amd64.deb ...
Unpacking libpng16-16:amd64 (1.6.34-1ubuntu0.18.04.2) ...
Selecting previously unselected package libfreetype6:amd64.
Preparing to unpack .../09-libfreetype6_2.8.1-2ubuntu2_amd64.deb ...
Unpacking libfreetype6:amd64 (2.8.1-2ubuntu2) ...
Selecting previously unselected package ucf.
Preparing to unpack .../10-ucf_3.0038_all.deb ...
Moving old data out of the way
Unpacking ucf (3.0038) ...
Selecting previously unselected package fonts-dejavu-core.
Preparing to unpack .../11-fonts-dejavu-core_2.37-1_all.deb ...
Unpacking fonts-dejavu-core (2.37-1) ...
Selecting previously unselected package fontconfig-config.
Preparing to unpack .../12-fontconfig-config_2.12.6-0ubuntu2_all.deb ...
Unpa

Selecting previously unselected package libavahi-client3:amd64.
Preparing to unpack .../31-libavahi-client3_0.7-3.1ubuntu1.2_amd64.deb ...
Unpacking libavahi-client3:amd64 (0.7-3.1ubuntu1.2) ...
Selecting previously unselected package libcups2:amd64.
Preparing to unpack .../32-libcups2_2.2.7-1ubuntu2.8_amd64.deb ...
Unpacking libcups2:amd64 (2.2.7-1ubuntu2.8) ...
Selecting previously unselected package libcupsimage2:amd64.
Preparing to unpack .../33-libcupsimage2_2.2.7-1ubuntu2.8_amd64.deb ...
Unpacking libcupsimage2:amd64 (2.2.7-1ubuntu2.8) ...
Selecting previously unselected package libijs-0.35:amd64.
Preparing to unpack .../34-libijs-0.35_0.35-13_amd64.deb ...
Unpacking libijs-0.35:amd64 (0.35-13) ...
Selecting previously unselected package libjbig2dec0:amd64.
Preparing to unpack .../35-libjbig2dec0_0.13-6_amd64.deb ...
Unpacking libjbig2dec0:amd64 (0.13-6) ...
Selecting previously unselected package libpaper1:amd64.
Preparing to unpack .../36-libpaper1_1.1.24+nmu5ubuntu1_amd64.deb 

Setting up libxml2:amd64 (2.9.4+dfsg1-6.1ubuntu1.3) ...
Setting up libfreetype6:amd64 (2.8.1-2ubuntu2) ...
Setting up fonts-noto-mono (20171026-2) ...
Setting up libgraphite2-3:amd64 (1.3.11-2) ...
Setting up libilmbase12:amd64 (2.2.0-11ubuntu2) ...
Setting up liblqr-1-0:amd64 (0.4.2-2.1) ...
Setting up libjbig2dec0:amd64 (0.13-6) ...
Setting up libpixman-1-0:amd64 (0.34.0-2) ...
Setting up libglib2.0-data (2.56.4-0ubuntu0.18.04.6) ...
Setting up krb5-locales (1.16-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libapparmor1:amd64 (2.12-4ubuntu5.1) ...
Setting up libltdl7:amd64 (2.4.6-2) ...
Setting up libijs-0.35:amd64 (0.35-13) ...
Setting up shared-mime-info (1.9-2) ...
Setting up libthai-data (0.1.27-2) ...
Setting up libxdmcp6:amd64 (1:1.1.2-3) ...
Setting up libkeyutils1:amd64 (1.5.9-9.2ubuntu2) ...
Setting up hicolor-icon-theme (0.17-2) ...
Setting up xdg-user-dirs (0.17-1ubuntu1) ...
Setting up libx11-data (2:1.6.4-3ubuntu0.2) ...
Setting up libx

update-alternatives: using /usr/bin/display-im6.q16 to provide /usr/bin/display-im6 (display-im6) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/display-im6.1.gz because associated file /usr/share/man/man1/display-im6.q16.1.gz (of link group display-im6) doesn't exist
update-alternatives: using /usr/bin/montage-im6.q16 to provide /usr/bin/montage (montage) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/montage.1.gz because associated file /usr/share/man/man1/montage-im6.q16.1.gz (of link group montage) doesn't exist
update-alternatives: using /usr/bin/montage-im6.q16 to provide /usr/bin/montage-im6 (montage-im6) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/montage-im6.1.gz because associated file /usr/share/man/man1/montage-im6.q16.1.gz (of link group montage-im6) doesn't exist
update-alternatives: using /usr/bin/mogrify-im6.q16 to provide /usr/bin/mogrify (mogrify) in auto mode
update-

In [14]:
!docker run --rm -v $(pwd)/testimages:/input -u $(id -u):$(id -g) image-invert
!ls -l testimages/inverted

total 368
-rw-r--r-- 1 localek10 bioeng  30553 Jun 15 23:49 cat00.jpg
-rw-r--r-- 1 localek10 bioeng  12352 Jun 15 23:49 cat01.jpg
-rw-r--r-- 1 localek10 bioeng 107932 Jun 15 23:49 cat02.jpg
-rw-r--r-- 1 localek10 bioeng 117656 Jun 15 23:49 cat03.jpg
-rw-r--r-- 1 localek10 bioeng  96046 Jun 15 23:49 cat04.jpg


### Exercise 2:

The docker command provides mechanisms for managing images and containers. By now you'll have at least one container that's been run, if you used `--rm` with your run commands they'll be stopped once the command completes but will persist. If something is still running in the background (perhaps your `hello-flask` server you made from the notes then forgot about) you can see this with `docker ps`:

```sh
$ docker ps
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
1cdc1e2ee834        hello-flask         "flask run --host=0.…"   29 hours ago        Up 29 hours                             quizzical_lichterman

```

The docker command line documentation is [here](https://docs.docker.com/engine/reference/commandline/cli) specifically covering this command and others. In this exercise  find the commands to kill any running containers you don't want anymore and clean up at finished containers kicking around. Images that aren't being used can also be cleaned up if you have any.

In [13]:
%%bash

docker ps
echo ---
docker container ls -a
echo ---
docker image ls -a
docker kill frosty_leavitt

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
d267d67d897a        hello-flask         "flask run --host=0.…"   3 seconds ago       Up 2 seconds        0.0.0.0:5000->5000/tcp   frosty_leavitt
---
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                      PORTS                    NAMES
d267d67d897a        hello-flask         "flask run --host=0.…"   3 seconds ago       Up 2 seconds                0.0.0.0:5000->5000/tcp   frosty_leavitt
77b534cb9ed8        310c494584e2        "/bin/sh -c 'adduser…"   5 minutes ago       Exited (1) 5 minutes ago                             wizardly_haslett
ba38a88c9ec1        310c494584e2        "/bin/sh -c 'adduser…"   5 minutes ago       Exited (1) 5 minutes ago                             xenodochial_newton
a9b5b5a88055        310c494584e2        "/bin/sh -c 'adduser…"   6 minutes ago       Exited (1) 6 minutes 